In [6]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [7]:
from helpers import load_csv_data

DATA_TRAIN_PATH = 'data/train.csv'

y, X, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)
print(y.shape, X.shape)

(5000,) (5000, 30)


## Prepare cost and prediction functions

In [8]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    os = np.ones(len(y))
    a = y * X.dot(w)
    b = (lambda_ / 2) * np.square(np.linalg.norm(w)) * os
    res = os - a
    res[np.where(res < 0)] = 0
    return np.sum(res + b)

In [9]:
from helpers import predict_labels

def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    pred = predict_labels(w, X)
    corr = y * pred
    return np.sum(corr[np.where(corr > 0)]) / len(y)

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [10]:
def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    x_n, y_n = X[n], y[n]
    a = 1 - y_n * x_n.dot(w)
    alpha = 1 if a > 0 else 0
    
    return -alpha * y_n * x_n + lambda_ * w

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [11]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 100000
    gamma = 0.01
    lambda_ = 0.001
    
    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=181703641.49754053
iteration=10000, cost=1457814.1385730272
iteration=20000, cost=630997.7206310771
iteration=30000, cost=168912.08150423912
iteration=40000, cost=90327.71009782818
iteration=50000, cost=81016.34271878388
iteration=60000, cost=73153.23944485391
iteration=70000, cost=68132.51263055197
iteration=80000, cost=62088.525367301896
iteration=90000, cost=58326.46830724351
training accuracy = 0.7124


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [60]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n, Q):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    # calculate the update of coordinate at index=n.
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    
    #print(alpha[n])
    alpha[n] = (lambda_ - (Q[n].dot(alpha) - Q[n,n] * old_alpha_n)) / Q[n,n]
    #print("NEW{}".format(alpha[n]))
    w = (1 / lambda_) * np.transpose(X).dot(np.diag(y)).dot(alpha)
    return w, alpha

In [61]:
def calculate_dual_objective(y, X, w, alpha, lambda_, Q):
    """calculate the objective for the dual problem."""
    a = np.transpose(alpha).dot(np.ones(len(y)))
    Y = np.diag(y)
    k = 1 / (2*lambda_)
    b = np.transpose(alpha).Q.dot(alpha)
    return a - k * b

In [63]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 100000
    lambda_ = 0.01

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    Q = np.diag(y).dot(X).dot(np.transpose(X)).dot(np.diag(y)) 
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n, Q)
            
        if it % 100 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_, Q)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

KeyboardInterrupt: 